# Data Wrangling

In [18]:
# This is a copy of ./predictions.ipynb for the second chance bracket.
# Changes will be marked with "V2"

from io import StringIO
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Options to display more/less rows in output
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_rows', 60)

# V2
random_seed = 42

# Past data (in csv)
team_stats = pd.read_csv(f'./data/cbb.csv')


# Fill empty values (teams didn't make tournament)
team_stats['POSTSEASON'] = team_stats['POSTSEASON'].fillna('NIT')
team_stats['SEED'] = team_stats['SEED'].fillna(-1)
team_stats_save_team_conf = team_stats.copy()

# Encode strings into numerical values for classifier
# Store team names/conferences as dictionary mapping to integers (numbers needed for RandomForest)
team_dict = {}
conf_dict = {}
def getFromDict(x, dict):
    if x not in dict:
        dict[x] = len(dict)
    return dict[x]
team_stats['TEAM'] = team_stats['TEAM'].apply(lambda x: getFromDict(x, team_dict))
team_stats['CONF'] = team_stats['CONF'].apply(lambda x: getFromDict(x, conf_dict))


# Current data, scrape data from barttorvik.com and read into dataframe
soup = BeautifulSoup(requests.get('https://barttorvik.com/trank.php#').content, "html.parser")

# Some spans contain seed, save the team/seed
tds = soup.findAll('td', {'class': 'teamname'})
seeds = {}
for td in tds:
    matches = re.match(r'(.+)\s+(\d+) seed, ✅', td.text)
    if matches:
        seeds[matches.groups()[0].replace('\xa0\xa0', '')] = matches.groups()[1]

# Spans contain extra data (not always seed), they can be removed:
for data in soup(['span']):
    data.decompose()
df = pd.read_html(StringIO(str(soup.find("table"))), header=1)[0]

# Remove header rows, match columns and merge into existing dataframe (there's probably an easier way to get columns to match, oh well 🤷‍♂️)
stats_24 = df[df['Rk'] != 'Rk'].drop(['Rk', '3PR', '3PRD'], axis=1).rename(columns={"EFG%": "EFG_O", "EFGD%": "EFG_D", "2P%": "2P_O", "2P%D": "2P_D", "3P%": "3P_O", "3P%D": "3P_D", "Adj T.": "ADJ_T", "Rec": "W"})
stats_24 = stats_24.replace('^(\d+)-(\d+)', r'\1', regex=True).replace('\+', '', regex=True)
stats_24.columns = map(str.upper, stats_24.columns)

# # Set seed (fill with n/a) and year (2024)
def setSeed(x):
    if x in seeds:
        return seeds[x]
    return -1
stats_24['SEED'] = stats_24['TEAM'].apply(lambda x: setSeed(x))
stats_24['YEAR'] = 2024
stats_24_save_team_conf = stats_24.copy()

# Use previously calculated team and conference values using dictionary
stats_24['TEAM'] = stats_24['TEAM'].apply(lambda x: getFromDict(x, team_dict))
stats_24['CONF'] = stats_24['CONF'].apply(lambda x: getFromDict(x, conf_dict))

team_stats

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,0,0,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,1,1,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,...,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,2,1,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,...,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,3,2,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,...,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,4,3,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,...,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3518,196,21,34,27,119.9,109.6,0.7369,56.3,52.9,13.6,...,27.5,54.6,52.1,39.7,36.1,69.5,-1.2,NIT,-1.0,2023
3519,117,11,33,27,111.4,97.3,0.8246,55.5,49.3,16.0,...,27.8,56.4,48.6,36.4,33.6,64.4,-2.0,NIT,-1.0,2023
3520,342,32,34,28,107.1,94.6,0.8065,51.7,44.0,19.3,...,28.7,52.5,42.8,33.4,31.1,69.8,-0.3,NIT,-1.0,2023
3521,157,16,38,29,112.4,97.0,0.8453,50.3,47.3,17.3,...,28.9,48.8,47.2,35.6,31.6,70.7,-0.5,NIT,-1.0,2023


# Create Train/Test Data and Classifier

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Split train/test
X = team_stats.drop('POSTSEASON', axis=1)
y = team_stats['POSTSEASON']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# V2
clf = RandomForestClassifier(random_state=random_seed)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# Analyze Classifier Performance

In [20]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.902127659574468


# Predict 2024 March Madness

Using the same process to make the classifier above, we can fit using all previous years data and predict against values in the `stats_2024` variable. We can then use the predicted `POSTSEASON` value for each team to determine the winner of each matchup (using the probability as the tiebreaker).

In [21]:
clf.fit(X, y)

# Reverse team_dict to get team names back from ids
reversed_team_dict = {}
for i in team_dict.keys():
  reversed_team_dict[team_dict[i]] = i
reversed_team_dict

# Find most likely POSTSEASON outcome, returning the team name, the outcome and its probability 
def getClassAndProb(x, idx):
  max_prob = 0
  max_idx = -1
  for i in range(len(x)):
    if x[i] > max_prob:
      max_prob = x[i]
      max_idx = i
  return [reversed_team_dict[idx], clf.classes_[max_idx], max_prob]

# Set probabilities
stats_with_predictions = stats_24.copy()
probabilites = clf.predict_proba(stats_24)
stats_with_predictions['TEAM'] = stats_with_predictions['TEAM'].apply(lambda x: reversed_team_dict[x])
stats_with_predictions['2ND'] = probabilites[:,0]
stats_with_predictions['Champions'] = probabilites[:,1]
stats_with_predictions['E8'] = probabilites[:,2]
stats_with_predictions['F4'] = probabilites[:,3]
stats_with_predictions['NIT'] = probabilites[:,4]
stats_with_predictions['R32'] = probabilites[:,5]
stats_with_predictions['R64'] = probabilites[:,6]
stats_with_predictions['R68'] = probabilites[:,7]
stats_with_predictions['S16'] = probabilites[:,8]
stats_with_predictions = stats_with_predictions[stats_with_predictions['SEED'] != -1]

stats_with_predictions

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,YEAR,2ND,Champions,E8,F4,NIT,R32,R64,R68,S16
0,Houston,2,36,32,120.2,86.5,.9778,50.5,43.9,13.7,...,2024,0.01,0.08,0.12,0.05,0.01,0.23,0.05,0.00,0.45
1,Connecticut,5,36,33,127.1,92.7,.9741,57.2,44.7,14.7,...,2024,0.07,0.09,0.26,0.02,0.00,0.32,0.08,0.00,0.16
2,Purdue,1,35,31,126.8,94.9,.9655,56.2,47.3,16.3,...,2024,0.05,0.08,0.14,0.04,0.00,0.17,0.33,0.00,0.19
3,Iowa St.,2,36,29,114.1,87.4,.9553,52.2,47.3,15.5,...,2024,0.01,0.04,0.16,0.01,0.00,0.25,0.10,0.00,0.43
4,Arizona,7,35,27,121.0,93.0,.9538,54.9,48.2,16.1,...,2024,0.08,0.01,0.14,0.03,0.00,0.32,0.17,0.00,0.25
6,Tennessee,4,34,26,115.3,90.2,.9438,51.3,45.0,14.9,...,2024,0.00,0.00,0.05,0.03,0.00,0.41,0.16,0.00,0.35
7,Duke,0,34,26,121.8,95.9,.9403,55.3,48.7,14.2,...,2024,0.05,0.01,0.06,0.02,0.00,0.32,0.26,0.00,0.28
8,North Carolina,0,36,29,118.4,93.9,.9355,51.7,46.4,14.1,...,2024,0.08,0.01,0.08,0.03,0.00,0.23,0.17,0.01,0.39
9,Marquette,5,36,27,119.3,95.2,.9308,55.5,49.7,14.3,...,2024,0.01,0.00,0.10,0.07,0.01,0.49,0.13,0.00,0.19
10,Gonzaga,3,33,27,123.0,98.3,.9293,57.1,46.8,14.1,...,2024,0.03,0.01,0.07,0.04,0.01,0.24,0.23,0.01,0.36


In [41]:
# Run through each matchup

# Find the winner of a matchup, compare teams' probabilities at current round (0 indexed with Round of 68)
team_1 = 'North Carolina St.'
team_2 = 'Connecticut'
current_round = 6

team_1_stats = stats_with_predictions[stats_with_predictions['TEAM'] == team_1]
team_2_stats = stats_with_predictions[stats_with_predictions['TEAM'] == team_2]


def getWinner(probabilites1, probabilites2, curr_round):
  sum_1 = 0
  sum_2 = 0
  for i in range(curr_round + 1):
    sum_1 += probabilites1[i]
    sum_2 += probabilites2[i]

  # The team with a higher probability at lower rounds is less likely to win:
  if sum_1 > sum_2:
    return 2
  return 1

team_1_probabilities = [team_1_stats['R68'].iloc[0], team_1_stats['R64'].iloc[0], team_1_stats['R32'].iloc[0], team_1_stats['S16'].iloc[0], team_1_stats['E8'].iloc[0], team_1_stats['F4'].iloc[0], team_1_stats['Champions'].iloc[0]]
team_2_probabilities = [team_2_stats['R68'].iloc[0], team_2_stats['R64'].iloc[0], team_2_stats['R32'].iloc[0], team_2_stats['S16'].iloc[0], team_2_stats['E8'].iloc[0], team_2_stats['F4'].iloc[0], team_2_stats['Champions'].iloc[0]]
team_1 if getWinner(team_1_probabilities, team_2_probabilities, current_round) == 1 else team_2

'North Carolina St.'

# Final Bracket Prediction

![Justin Thoms - ESPN March Madness 2024 Second Chance Bracket: Winner North Carolina State](SecondChanceBracket.png)